
<center><h1>Interactive Visualization Report</h1></center>
<center><h3>Weiping Zhang</h3></center>
<center><h3>FHNW    data Science</h3></center>
<center><h3>2024    HS</h3></center>

In [1]:
# load libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import tracemalloc
from IPython.display import IFrame
# import nbformat
import itertools
# import dash
from dash import Dash, dcc, html, Input, Output
from dash.dependencies import Input, Output, State
import plotly.express as px
# from werkzeug.middleware.profiler import ProfilerMiddleware
import time
from threading import Timer
from plotly.subplots import make_subplots 
import webbrowser
from plotly.colors import qualitative# , sample_colorscale


# 1 Performance

### Requirements

The larger the data, containing more variables, the more we need interactive visualizations. Both the data size and the interactivity can create performance issues.

You will learn 
-   why an interaction visualization may be slow (e.g., how much data is too much, what hardware and bandwidth does one need to ensure that the visualization works smoothly), and learn about 
-   the current solutions e.g., tiling, level of detail management, using GPU, how can you prioritize and hierarchically organize the data. 
-   For your hands-on exercise, you can 
    -   implement one of the performance enhancing solutions, 
    -   and/or you can demonstrate how computational performance is affected by benchmarking rendering or streaming time e.g., by varying input data size, visualization type, or with vs. without a performance management intervention. 
    -   At the end of this, you should understand the limits of your data and tools, and make good decisions for a performant visualization.

In [2]:
# load rte
rte = pd.read_csv('data/rte_fr_generation_2015_2023.csv').drop(columns=['Unnamed: 0']).rename(columns={'value': 'Electricity Generation (MWh)'})
# remove the 'TOTAL' category from production_type
rte = rte[rte['production_type'] != 'TOTAL']
# start date precise on hour, day, month and year
rte['date'] = pd.to_datetime(rte['start_date'].apply(lambda x: x[:-6]))
rte

,production_type,start_date,end_date,Electricity Generation (MWh),date
0,BIOMASS,2015-01-01T00:00:00+01:00,2015-01-01T01:00:00+01:00,193,2015-01-01 00:00:00
1,BIOMASS,2015-01-01T01:00:00+01:00,2015-01-01T02:00:00+01:00,193,2015-01-01 01:00:00
2,BIOMASS,2015-01-01T02:00:00+01:00,2015-01-01T03:00:00+01:00,193,2015-01-01 02:00:00
3,BIOMASS,2015-01-01T03:00:00+01:00,2015-01-01T04:00:00+01:00,193,2015-01-01 03:00:00
4,BIOMASS,2015-01-01T04:00:00+01:00,2015-01-01T05:00:00+01:00,193,2015-01-01 04:00:00
...,...,...,...,...,...
866432,WIND_ONSHORE,2023-05-27T19:00:00+02:00,2023-05-27T20:00:00+02:00,2394,2023-05-27 19:00:00
866433,WIND_ONSHORE,2023-05-27T20:00:00+02:00,2023-05-27T21:00:00+02:00,2749,2023-05-27 20:00:00
866434,WIND_ONSHORE,2023-05-27T21:00:00+02:00,2023-05-27T22:00:00+02:00,4261,2023-05-27 21:00:00
866435,WIND_ONSHORE,2023-05-27T22:00:00+02:00,2023-05-27T23:00:00+02:00,5313,2023-05-27 22:00:00


In [3]:
rte_day = rte.copy()
rte_day['date'] = pd.to_datetime(rte['date']).dt.date
rte_day = rte_day.groupby(['production_type','date']).agg({'Electricity Generation (MWh)':'sum'}).reset_index()

rte_month = rte.copy()
rte_month['date'] = pd.to_datetime(rte['date']).dt.to_period('M').astype(str)
rte_month = rte_month.groupby(['production_type','date']).agg({'Electricity Generation (MWh)':'sum'}).reset_index()
rte.shape, rte_day.shape, rte_month.shape

((794899, 5), (33139, 3), (1112, 3))

In [4]:
# Fixed the color for each category in the production_type column
default_colors = px.colors.qualitative.Plotly
categories = rte['production_type'].unique()  
color_map = {category: color for category, color in zip(categories, itertools.cycle(default_colors))}

In [5]:
def performance_measure(df, date_type, n, m, plot_type='line'):
    """date_type: 'hour', 'day', 'month'"""
    rendering_time, memory_usage = [], [] 
    for _ in range(n):  # Repeat n times

        # start tracking rendering time and memory
        start_time = time.time()  # Start timing before plot creation
        tracemalloc.start()
        
        for i in range(m):  # Render m plots
            # Create the figure (rendered in memory but not displayed or exported)
            if plot_type == 'line':
                fig = px.line(df, x='date', y='Electricity Generation (MWh)', color='production_type', color_discrete_map=color_map, labels={'date': 'Date'}
                       ).update_layout(title=f"Electricity Generation per {date_type}")
            elif plot_type == 'heatmap':
                fig = px.imshow(df.pivot(index='production_type', columns='date', values='Electricity Generation (MWh)'),
                    title=f"Figure: Electricity Generation per {date_type}",
                    labels={'x': 'Date', 'y': 'Production Type', 'color': 'Electricity Generation (MWh)'})

        # Stop tracking rendering time and memory
        end_time = time.time() 
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        # Display the first plot
        if _==0:
            fig.show()
        rendering_time.append(end_time - start_time) 
        memory_usage.append(peak / 1024 / 1024)  # Convert to MB
    return rendering_time, memory_usage

In [7]:
%%skip
# this result is calculated on an seven-year old laptop with intel core-i5 cpu.
data_rendering_time_i5 = [[108.06678276062011,
 156.1674476623535,
 132.48159236907958,
 174.6815710067749,
 120.68382549285889,
 129.62000217437745,
 133.02882537841796,
 126.17757663726806,
 123.90179901123047,
 132.8648422241211,
 133.3771474838257,
 139.26874923706055,
 122.04780216217041,
 122.03372840881347,
 124.67757244110108,
 121.85608654022217,
 130.95600337982177,
 141.47163677215576,
 134.9555679321289,
 136.88130378723145],
 [6.669061851501465,
 7.3801929473876955,
 19.309012794494627,
 6.778440666198731,
 7.511601066589355,
 8.017717742919922,
 6.138898658752441,
 6.735079956054688,
 6.156294250488282,
 6.538073348999023,
 6.144527244567871,
 6.770983695983887,
 6.297557830810547,
 7.348487854003906,
 6.164977073669434,
 6.827363586425781,
 6.236383438110352,
 6.50987434387207,
 6.210908699035644,
 6.540999984741211],
 [1.3284378051757812,
 1.7937156677246093,
 1.3633726119995118,
 1.4009876251220703,
 1.795792579650879,
 1.5447874069213867,
 1.4393318176269532,
 1.3779693603515626,
 1.3803293228149414,
 1.3529108047485352,
 1.399618911743164,
 1.4152471542358398,
 1.508279037475586,
 1.6435300827026367,
 1.5708909988403321,
 1.4851293563842773,
 1.3771402359008789,
 1.3662988662719726,
 1.5093027114868165,
 1.3839767456054688]]
data_memory_usage_i5 = [[355.6098831176758,
 348.01308212280276,
 367.7540756225586,
 311.48339996337893,
 339.89624099731446,
 340.02167053222655,
 304.2357666015625,
 324.00997161865234,
 305.95603637695314,
 339.9346473693848,
 340.02415161132814,
 371.03240356445315,
 339.9120933532715,
 347.9682052612305,
 367.719229888916,
 313.88771896362306,
 367.8483345031738,
 311.36732177734376,
 340.0095489501953,
 339.90820388793946],
 [9.907402801513673,
 10.211827850341797,
 10.185945892333985,
 10.095604705810548,
 10.119969940185547,
 10.236217498779297,
 10.225820159912109,
 10.230951690673828,
 10.529962921142578,
 10.219470977783203,
 10.537568664550781,
 10.217238616943359,
 10.108668518066406,
 9.982584381103516,
 10.109043884277344,
 10.212093353271484,
 9.905667114257813,
 9.982376098632812,
 10.233802795410156,
 10.123918914794922],
 [1.2100944519042969,
 1.3352149963378905,
 1.1147857666015626,
 1.2060935974121094,
 0.9346328735351562,
 1.1983360290527343,
 1.1917327880859374,
 1.0585029602050782,
 1.2394378662109375,
 1.0993576049804688,
 1.2189041137695313,
 1.2054336547851563,
 1.106768798828125,
 1.226092529296875,
 1.096623992919922,
 1.0764900207519532,
 1.1998329162597656,
 1.0857887268066406,
 1.1916305541992187,
 1.0576194763183593]]

UsageError: Cell magic `%%skip` not found.


In [8]:
%%skip
# this cell will be skiped, as the calculation on the rendering time and memory usage for the Apple M2 chip is slow.
# if you want to execute this cell, please uncomment the code below
# Computing performance rendering time for different datasizes
# 20 repetitions, 4 plots per repetition
n, m = 20, 4

rendering_time1, memory_usage1 = performance_measure(rte, 'hour', n, m)
rendering_time2, memory_usage2 = performance_measure(rte_day, 'day', n, m)
rendering_time3, memory_usage3 = performance_measure(rte_month, 'month', n, m)

# Data for boxplots
data_rendering_time_m2 = [rendering_time1, rendering_time2, rendering_time3]
data_memory_usage_m2 = [memory_usage1, memory_usage2, memory_usage3]
cpus = ["Apple M2", "Intel Core i5"]
paddings = [8, 70]
for m in range(2):
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
    data_rendering_time = [data_rendering_time_m2, data_rendering_time_i5][m]
    data_memory_usage = [data_memory_usage_m2, data_memory_usage_i5][m]

    # Left plot: Rendering time
    axes[0].boxplot(data_rendering_time, labels=['Large', 'Medium', 'Small'])
    axes[0].set_title('')
    axes[0].set_xlabel('Data Size')
    axes[0].set_ylabel('Rendering Time (s)')

    # Calculate maximum value for y-axis adjustment
    max_rendering_time = max(max(dataset) for dataset in data_rendering_time)
    axes[0].set_ylim(0, max_rendering_time + paddings[m])  # Add padding to y-axis

    # Add median annotations for Rendering Time (on top of the box)
    for i, dataset in enumerate(data_rendering_time, start=1):  # Box indices start at 1
        median = np.median(dataset)
        axes[0].text(i, median + paddings[m], f'{median:.2f}', ha='center', va='bottom', fontsize=10, color='black')  # Adjusted y-coordinate

    # Right plot: Memory Usage
    axes[1].boxplot(data_memory_usage, labels=['Large', 'Medium', 'Small'])
    axes[1].set_title('')
    axes[1].set_xlabel('Data Size')
    axes[1].set_ylabel('Memory Storage (MB)')

    # Calculate maximum value for y-axis adjustment
    max_memory_usage = max(max(dataset) for dataset in data_memory_usage)
    axes[1].set_ylim(0, max_memory_usage + 65)  # Add padding to y-axis

    # Add median annotations for Memory Usage (on top of the box)
    for i, dataset in enumerate(data_memory_usage, start=1):  # Box indices start at 1
        median = np.median(dataset)
        axes[1].text(i, median + 60, f'{median:.2f}', ha='center', va='bottom', fontsize=10, color='black')  # Adjusted y-coordinate

    fig.suptitle(f'Rendering Time & Memory Usage using {cpus[m]}', fontsize=14)
    plt.tight_layout()
    plt.show()


UsageError: Cell magic `%%skip` not found.


# LO2: Dashboard design principles

Interactive visualization often means working with the concept of a ‘dashboard’. In these dashboards user typically has **access to various controls and multiple linked visualizations**. 
-   How to **design linked views**, or 
-   **focus+context displays**? 
-   What are **brushing and linking**? 
-   What is **"Shneiderman's mantra**: 
    -   Overview first, zoom and filter, details on demand" and how can you apply this principle in an interactive visualization?

In [9]:
df_total = pd.read_csv('data/de_at_fr_it_type_generation_emission_2015_2023.csv').drop(columns=['gCO2eq/kWh']) # [['date', 'total_gCO2eq', "gCO2eq/kWh", 'total_generation(MWh)', 'country']]
print(f"Before rte reduction, the rte has {df_total.shape[0]} rows and {df_total.shape[1]} columns")
df_total.head()

Before rte reduction, the rte has 101060 rows and 6 columns


,date,source,total_generation(MWh),proportion_electr_generation,total_gCO2eq,country
0,2015-01-01,coal,257344.00,17.766226,2.779315e+11,Germany
1,2015-01-02,coal,296727.50,21.940168,3.204657e+11,Germany
2,2015-01-03,coal,287900.50,22.720985,3.109325e+11,Germany
3,2015-01-04,coal,405171.25,27.784973,4.375850e+11,Germany
4,2015-01-05,coal,423603.75,28.743335,4.574920e+11,Germany


In [10]:
df_co2_intensity = pd.read_csv('data/co2_intensity.csv', sep = ',').dropna(axis=1)
df_co2_intensity = df_co2_intensity[df_co2_intensity['source'].isin(df_total.source.unique())].reset_index(drop=True)

- The original rteset includes the electricity total generation per day per country across eight years from 2015 till 2023. Intotal, we get 11031 rte points. 
- This maybe not yet too large, however, when including multiple interactive plots in a dashboard, it may be computational expensive to process and negatively affact reaction speed.

I would reduce the rteset from per day to per month, which means adding up the generation amount within one month for each country.

In [11]:
df_import = pd.read_csv('data/import_data.csv', sep = ',').dropna(axis=1).iloc[:, 1:].rename(columns={'Date': 'date'})
df_import = df_import[df_import['country'] != 'Liechtenstein']
df_import = df_import[df_import['date'] >= '2015-01-01']
df_import

,date,country,import_quantity(MWh)
180,2015-01-01,Germany,1926
181,2015-02-01,Germany,1750
182,2015-03-01,Germany,1536
183,2015-04-01,Germany,1113
184,2015-05-01,Germany,266
...,...,...,...
1107,2022-10-01,Austria,116
1108,2022-11-01,Austria,512
1109,2022-12-01,Austria,519
1110,2023-01-01,Austria,634


In [12]:
df_total = df_total.merge(df_co2_intensity, on=['source'], how='left')

categories = df_co2_intensity.sort_values(by='gCO2eq/kWh', ascending=False)['source'].unique()  
# Assign a unique color to each category
color_map_source = {category: color for category, color in zip(categories, itertools.cycle(qualitative.Plotly + qualitative.Pastel))}


# LO3 HCI Basics


In an interactive visualization, it is of great importance that a user can navigate the interface and interaction techniques / paradigms. You will learn some of the technology and basics of how to design an interactive visualization especially in the presence of multiple linked visualizations on a dashboard as explained above: 

-   What are good ways to utilize basic interactive functions such as select, zoom and filter? 
-   What is your main input modality such as mouse, keyboard, touch … etc? 
-   Is your visualization responsive to all hardware or are you designing for a specific one e.g., a smart phone / tablet, laptop/PC, extended (virtual, augmented, mixed) reality? 
-   What advanced interaction techniques are interesting (i.e., could gesture or voice input be of use in visualization use)?

You can design your UI (user interface) based on a **cognitive walk-through** for your own project, see if you can find some references e.g. using google scholar to support your choices. Last but not least, you can validate your choices in LO4.

Additionally you will also learn strategies such as animated transitions and breadcrumbs to facilitate navigation and reflect on how these may be useful for your project.

In [13]:
# convert data from daily to monthly
df_total_red = df_total.copy()
df_total_red['date'] = pd.to_datetime(df_total_red['date']).dt.to_period('M').astype(str)

df_total_red.groupby(['date', 'source', 'country']).agg({
    'total_generation(MWh)': 'sum',
    'total_gCO2eq': 'sum',
    'gCO2eq/kWh': 'first'
}).reset_index()

df_total_red['total_tCO2eq'] = df_total_red['total_gCO2eq']/1000000
df_import['date'] = pd.to_datetime(df_import['date']).dt.to_period('M').astype(str)


all_dates = sorted(pd.concat([df_total_red['date'], df_import['date']]).unique())
initial_start = all_dates[0]
initial_end = all_dates[-1]

def make_figure1(start_date, end_date, df):
    fig = px.line(
        df,
        x='date',
        y='import_quantity(MWh)',
        color='country',
        title='Electricity Import Trends by Country',
        labels={'date': '', 'import_quantity(MWh)': 'Import Quantity (MWh)'})
    fig.update_layout(
        title={
            'x': 0.5,  # Center the title horizontally
            'xanchor': 'center',  # Anchor the title at the center
            'yanchor': 'top'      # Optional: Anchor the title at the top
        },
        legend_title_text="Country"  # Set the legend title
    )
    # Limit x-axis to [start_date, end_date]:
    fig.update_xaxes(range=[start_date, end_date])
    return fig

def make_figure2(start_date, end_date, df):
    #df['date'] = pd.to_datetime(df['date'])
    start_date = pd.to_datetime(start_date)
    end_date   = pd.to_datetime(end_date)
    df = df[(pd.to_datetime(df['date']) >= start_date) & (pd.to_datetime(df['date']) <= end_date)]
  
    fig = px.pie(
        df,
        names='source',
        values='total_generation(MWh)',
        title='Import Electricity Composition by Source',
        color='source',
        color_discrete_map=color_map_source)
    fig.update_traces(
        hovertemplate="<b>%{label}</b>: %{value} MWh (%{percent})",
        textinfo='percent'  # Display percentage on the chart
    )
    fig.update_layout(title={
            'x': 0.5,  # Center the title horizontally
            'xanchor': 'center',  # Anchor the title at the center
            'yanchor': 'top'      # Optional: Anchor the title at the top
        },
        showlegend=False)
    # Limit x-axis to [start_date, end_date]:
    fig.update_xaxes(range=[start_date, end_date])
    return fig

def make_figure3(start_date, end_date, df):
    fig = px.bar(
        df[['source', 'gCO2eq/kWh']].drop_duplicates().sort_values(by='gCO2eq/kWh', ascending=False),
        y='source',
        x='gCO2eq/kWh',
        color='source',
        title='CO2 Intensity by Energy Source',
        labels={'gCO2eq/kWh': 'CO2 Intensity (gCO2eq/kWh)', 'source': ''},
        color_discrete_map=color_map_source
    )
    fig.update_layout(title={
            'x': 0.5,  # Center the title horizontally
            'xanchor': 'center',  # Anchor the title at the center
            'yanchor': 'top'      # Optional: Anchor the title at the top
        },
        showlegend=False)
    # Limit x-axis to [start_date, end_date]:
    fig.update_xaxes(range=[start_date, end_date])
    return fig

def make_figure4(start_date, end_date, df):
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
        subplot_titles=("Total Generation (MWh)", "Total CO2 Emissions (ton)")
    )

    # Define a consistent color map for countries
    country_colors = px.colors.qualitative.Plotly
    unique_countries = df['country'].unique()
    color_map_country = {country: country_colors[i % len(country_colors)] for i, country in enumerate(unique_countries)}

    # Add traces for total_generation(MWh)
    for country in unique_countries:
        country_data = df[df['country'] == country].groupby(['country', 'date']).agg({'total_generation(MWh)': 'sum'}).reset_index()

        fig.add_trace(
            go.Scatter(
                x=country_data['date'],
                y=country_data['total_generation(MWh)'],
                mode='lines',
                name=f"{country}",
                line=dict(color=color_map_country[country]),
                legendgroup=country
            ),
            row=1, col=1
        )

    # Add traces for total_tCO2eq
    for country in unique_countries:
        country_data = df[df['country'] == country].groupby(['country', 'date']).agg({'total_tCO2eq': 'sum'}).reset_index()
        fig.add_trace(
            go.Scatter(
                x=country_data['date'],
                y=country_data['total_tCO2eq'],
                mode='lines',
                name=f"{country}",
                line=dict(color=color_map_country[country]),
                legendgroup=country,
                showlegend=False  # Hide duplicate legend entries
            ),
            row=2, col=1
        )
    fig.update_layout(
        title={
            'text': "Electricity Generation and CO2 Emissions in Importing Countries",
            'x': 0.5,  # Center the title horizontally
            'xanchor': 'center',  # Anchor the title at the center
            'yanchor': 'top'      # Anchor the title at the top
        },
        height=600,  # Specify the height of the figure
        legend_title_text="Country",  # Update the legend title
        xaxis_title=""  # Update the x-axis title
        )
    # Limit x-axis to [start_date, end_date]:
    fig.update_xaxes(range=[start_date, end_date])
    return fig

# plots 
import_fig = make_figure1(initial_start, initial_end, df_import)
pie_fig = make_figure2(initial_start, initial_end, df_total_red)
bar_fig = make_figure3(initial_start, initial_end, df_total_red)
emission_fig = make_figure4(initial_start, initial_end, df_total_red)


app = Dash(__name__)
app.layout = html.Div([
    html.Div([
    html.H1("Switzerland Import Electricity Status", style={
        'text-align': 'center',
        'margin-top': '10px',
        'margin-bottom': '20px'
    }),
    

    # Filters and Reset Button in a Single Row
    html.Div([
        html.Label(" "),
        # Date Picker (Left)
        html.Div([
            
            dcc.DatePickerRange(
            id="date-picker",
            start_date=initial_start,
            end_date=initial_end,
            min_date_allowed=initial_start,
            max_date_allowed=initial_end,
            display_format="YYYY-MM",
            updatemode="bothdates"
        )
    ], style={
        'width': '30%', 
        'margin': '0 auto', 
        'padding-left': '200px', 
        'padding-right': '10px', 
        'margin-left': '20px', 
        'margin-right': '20px'
    }), 

        # Country Filter (right)
        html.Div([
            html.Label(" "),
            dcc.Dropdown(
                id='country-dropdown',
                options=[{'label': country, 'value': country} for country in df_total_red['country'].unique()],
                multi=True,
                value=df_total_red['country'].unique(),  # Default to all countries
                placeholder="Select countries"
            )
        ], style={'width': '30%', 'padding-right': '50px', 'margin-left': '20px', 'margin-right': '20px',}), 
        
    
        
        ], style={
            'display': 'flex',
            'margin': '0 auto', 
            'margin-bottom': '10px',
            'align-items': 'center'  # Align all items vertically
        })
    ], style={
        'position': 'fixed',  # Keep the filters and reset button fixed
        'justify-content': 'center',  # Align items horizontally at the center

        'top': '0',           # Align to the top of the page
        'width': '100%',      # Full width of the page
        'background-color': 'white',  # Add background color to prevent overlap
        'z-index': '1000',    # Ensure it stays on top of other elements
        'padding': '2px 0',  # Add padding for better spacing
        'box-shadow': '0px 4px 6px rgba(0, 0, 0, 0.1)'  # Optional shadow for separation
    }),

    # Main Content (Graphs)
    html.Div([
        # Add padding to compensate for the fixed header
        html.Div(style={'height': '100px'}),  # Spacer to avoid overlap with the fixed header

        # Graphs Layout: Left 60% and Right 40%
        html.Div([
            html.Div([
                dcc.Graph(id='line-plot-import', figure=import_fig,
                          config={'modeBarButtonsToRemove': ['toImage','autoScale2d','zoomIn2d','zoomOut2d'],
                        'displaylogo': False})  # This removes the Plotly logo  # Figure 1
            ], style={'width': '60%', 'display': 'inline-block', 'vertical-align': 'top'}),  # Left 60%

            html.Div([
                dcc.Graph(id='pie-plots',figure=pie_fig,
                          config={'modeBarButtonsToRemove': ['toImage', 'autoScale2d','zoomIn2d','zoomOut2d'],
                        'displaylogo': False})  # This removes the Plotly logo)  # Figure 2
            ], style={'width': '40%', 'display': 'inline-block', 'vertical-align': 'top'}),  # Right 40%
        ], style={'width': '100%', 'display': 'flex', 'justify-content': 'space-between', 'margin-bottom': '20px', 'margin-top': '30px',}),

        # Figures 3 and 4 side-by-side
        html.Div([
            html.Div([
                dcc.Graph(id='emission_fig',figure=emission_fig,
                          config={'modeBarButtonsToRemove': ['toImage', 'autoScale2d','zoomIn2d','zoomOut2d'],
                        'displaylogo': False})  # This removes the Plotly logo))  # Figure 4
            ], style={'width': '60%', 'display': 'inline-block', 'vertical-align': 'top'}),  # Right 40%

            html.Div([
                dcc.Graph(id='bar-plot-co2',figure=bar_fig,
                          config={'modeBarButtonsToRemove': ['toImage', 'autoScale2d','lasso2d', 'select2d','zoomIn2d','zoomOut2d'],
                        'displaylogo': False})  # This removes the Plotly logo)  # Figure 3
            ], style={'width': '40%', 'display': 'inline-block', 'vertical-align': 'top'}),  # Left 60%
        ], style={'width': '100%', 'display': 'flex', 'justify-content': 'space-between'})
    ], style={'width': '80%', 'margin': '0 auto'})  # Outer container
])


@app.callback(
    [
     
        Output('line-plot-import', 'figure'),
        Output('pie-plots', 'figure'),
        Output('bar-plot-co2', 'figure'),
        Output('emission_fig', 'figure'),
        Output("date-picker", "start_date"),
        Output("date-picker", "end_date"),
    ],
    [
        Input('country-dropdown', 'value'),
        Input('line-plot-import', 'relayoutData'),
        Input('emission_fig', 'relayoutData'),
        Input("date-picker", "start_date"),
        Input("date-picker", "end_date"),
    
    ],
    [
        State('line-plot-import', 'figure'),
        State('emission_fig', 'figure')
    ]
)



def update_figures(
    selected_countries,
    relayout_line,
    relayout_emission,
    dp_start, 
    dp_end,
    line_fig_state,
    emission_fig_state
):  
    # 1) Extract the current date range from the date picker
    start_date = pd.to_datetime(dp_start)
    end_date = pd.to_datetime(dp_end)


    # 2) Check if user zoomed/selected in graph-1
    #    relayoutData typically includes keys like 'xaxis.range[0]', 'xaxis.range[1]' if zoomed
    if relayout_line and "xaxis.range[0]" in relayout_line and "xaxis.range[1]" in relayout_line:
        start_date = pd.to_datetime(relayout_line["xaxis.range[0]"])
        end_date   = pd.to_datetime(relayout_line["xaxis.range[1]"])

    # 3) Check if user zoomed/selected in graph-2
    if relayout_emission and "xaxis.range[0]" in relayout_emission and "xaxis.range[1]" in relayout_emission:
        start_date = pd.to_datetime(relayout_emission["xaxis.range[0]"])
        end_date   = pd.to_datetime(relayout_emission["xaxis.range[1]"])

    # Update the displayed selected date range
    date_text = f"{start_date} to {end_date}"

    # Filter data by selected countries
    if selected_countries:
        filtered_import = df_import[df_import['country'].isin(selected_countries)]
        filtered_total = df_total_red[df_total_red['country'].isin(selected_countries)]

    # Line plot for df_import
    updated_import_fig = make_figure1(start_date, end_date, filtered_import)

    # Create Pie Plot
    updated_pie_fig = make_figure2(start_date, end_date, filtered_total)

    # Bar plot for df_co2_intensity
    updated_bar_fig = make_figure3(start_date, end_date, filtered_total)

    # Line plot with upper and lower parts
    updated_emission_fig = make_figure4(start_date, end_date, filtered_total)

  

    return updated_import_fig, updated_pie_fig, updated_bar_fig, updated_emission_fig, start_date, end_date

def open_browser():
    webbrowser.open_new("http://127.0.0.1:8003/")  # Update the URL if necessary

In [14]:
%%capture
if __name__ == "__main__":
    app.run_server(debug=True, host='0.0.0.0', port=8003, use_reloader=False)
    Timer(1, open_browser).start() 


In [15]:
IFrame(src="http://127.0.0.1:8003/", width="100%", height="1300px")
 

# LO4 Evaluation
-   User 1 (female, age 30 - 35, photographer): 5, 5, 5, 5, 3, 5, 4, 5, 4, 5, -, -
-   User 2 (male, age 35 - 40, teacher): 5, 4, 4, 5, 4, 5, 4, 4, 5, 5
-   User 3 (male, age 65 - 70, speech therapist): 4, 5, 2, 5, 5, 5, 5, 3, 5, 5

average of for each category: (4.33, 4.67, 4.17, 4.83)